In [2]:
import datetime
import time
import tweepy
import pandas as pd
import numpy as np 


# get_details function

In [36]:

consumer_key ="tH7AnhdvgOndxvZOyeeyfHOVG"
consumer_secret ="14mR3PZn3osH1OBPjtJWJT7IycQB5kOF4PmIuDjHvFHuEZppk0"
access_key ="2616201013-Bow3klvUxQbYXuO22nCm4tdH4Wxvsr6OE8PGXPT"
access_secret ="2iAs1oM1dhVoUX4gnRR9dGCPI4Ayc8ZbfJJymbqfm5otP"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)
def convert_twitter_datetime_to_string(date):
	return time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(date,'%a %b %d %H:%M:%S +0000 %Y'))

def convert_string_to_datetime(date):
	return datetime.datetime.strptime(date, "%Y-%m-%d %H:%M:%S")

def get_details(username):
	
	try:
		userdata=api.get_user(screen_name=username)
		
	except Exception as e:
		return 
		# 200 is the max 
	
	end_date = datetime.datetime.now()
	start_date = datetime.datetime.now() - datetime.timedelta(days=7)
	user = userdata._json
	num_of_retweets_by_user = 0
	num_of_tweets_this_week = 0
	monday_to_sunday = [0] * 7
	twelve_am_to_eleven_pm = [0] * 24
	retweeted=0
	most_recent_post=0
	tweet_language=''
	try:
		timeline = api.user_timeline(screen_name=username, count=200,include_rts=True, tweet_mode="extended")
		if len(timeline) >= 1:
			most_recent_post=convert_twitter_datetime_to_string(timeline[0]._json["created_at"])
			tweet_language=timeline[0]._json["lang"]
			for tweet in timeline:
				tweet = tweet._json
				retweeted+=tweet["retweet_count"]
				if ("retweeted_status" in tweet):
					num_of_retweets_by_user += 1
				tweet_time = convert_string_to_datetime(convert_twitter_datetime_to_string(tweet["created_at"]))
				if (tweet_time < end_date and tweet_time > start_date):
					num_of_tweets_this_week += 1
				monday_to_sunday[datetime.datetime.weekday(tweet_time)] += 1
				twelve_am_to_eleven_pm[tweet_time.hour] += 1
		else:
			pass
	except Exception as e:
		pass
	
	
	user_data = {
		"image": user["profile_image_url_https"],
		"background_image": user["profile_background_image_url"],
		"verified": user["verified"],
		"statuses": user["statuses_count"],
		"screen_name": user["screen_name"],
		"display_name": user["name"],
		"description": user["description"],
		"location": user["location"],
		"url": user["url"],
		"date_joined": convert_twitter_datetime_to_string(user["created_at"]),
		"most_recent_post": most_recent_post,
		"statuses": user["statuses_count"],
		"following": user["friends_count"],
		"followers": user["followers_count"],
		"likes": user["favourites_count"],
		"lists": user["listed_count"],
		"twitter_user_id": user["id_str"],
		"tweet_language": tweet_language,
		"tweets_this_week": num_of_tweets_this_week,
		"retweet_ratio": int(num_of_retweets_by_user / 2),
		"retweeted_count": int(retweeted),
		"tweets_by_day_of_week": monday_to_sunday,
		"tweets_by_hour_of_day": twelve_am_to_eleven_pm,
	}
	return (user_data)
print(get_details('BabyDogeCEX'))


{'image': 'https://pbs.twimg.com/profile_images/1490234654955380742/KSR1C5CT_normal.jpg', 'background_image': None, 'verified': False, 'statuses': 1325, 'screen_name': 'BabyDogeCEX', 'display_name': 'BabyDoge', 'description': 'BabyDogeCoin is a meme with a mission!🦴\nNot affiliated with @babydogecoin', 'location': 'United States', 'url': 'https://t.co/p1LHo6yVc8', 'date_joined': '2022-02-06 07:55:04', 'most_recent_post': '2023-02-20 10:12:11', 'following': 74, 'followers': 13275, 'likes': 254, 'lists': 14, 'twitter_user_id': '1490232458981388288', 'tweet_language': 'en', 'tweets_this_week': 46, 'retweet_ratio': 1, 'retweeted_count': 51839, 'tweets_by_day_of_week': [31, 29, 31, 26, 27, 30, 26], 'tweets_by_hour_of_day': [0, 0, 8, 4, 5, 7, 6, 5, 12, 8, 18, 13, 10, 8, 12, 17, 9, 37, 15, 4, 2, 0, 0, 0]}


# split data 

In [ ]:
df = pd.read_csv("./cleandata.csv")

groups = df.groupby(np.arange(len(df.index))//20)
i=0
for (frameno, frame) in groups:
    i+=1
    frame.to_csv("./twitterdataset/data/%s.csv" % frameno)

In [56]:
alldata=[]
nbr_req=0
data=pd.read_csv("./cleandata.csv")
for index, row in data.iterrows():
    nbr_req += 1
    u = []
    u=get_details(row['screen_name'])
    if u :
        u['account_type']=row['account_type']
        alldata.append(u)
    if nbr_req==900:
        print('req = 900 please wait 15 minutes')
        nbr_req=0
        time.sleep(900)
        print('15 minutes')


req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes
15 minutes
req = 900 please wait 15 minutes


In [40]:
alldata
alldata.to_json("./twitterdataset/data.json")

""
